In [6]:
import os

path = 'JOURNO/'
fns = [path + x for x in os.listdir(path) if x[:2] == 'jj']

In [19]:
bits = []
for fn in fns[:]:
    with open(fn,'r') as fp:
        dd = [fp.readline().strip() for x in range(3)]
    bits.append((dd[0],dd[2]) )

I scraped articles to a txt file, the format is as follows

- line 0 = title
- blank
- line 2 = contents of the meta i.e. kwords
- blank
- contents
- blank
- line -1 = src URL



In [21]:
titles, kwords = zip(* bits)
vocab = ','.join(kwords).split(',')

Given the format one can get the contents as follows

In [90]:
arts  = []
for i in bag:
    with open(fns[i],'r') as fp:
        dd = ''.join(fp.readlines()[4:-2]).replace('\n', ' ')
      
    arts.append(dd)

In [96]:

lengths = [len(x) for x in arts]
totals = [(sum(lengths[:i]),i)  for i,x in enumerate(lengths)]
totals[-1]

(789779, 164)

In [100]:
tx = ' '.join(arts)
with open('text6.txt', 'w') as fp:
    fp.write(tx)

In [33]:
from collections import Counter

word_count = Counter(vocab)
wc = sorted(list(word_count.items()), key = lambda x : -x[1] )

In [54]:
junk = ','.join(word_count.keys() )

In [48]:
import textacy

In [55]:
doc = textacy.doc.Doc(junk)

In [78]:
names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))

people = [x for x in names if x.label_ == 'PERSON' and len(x) > 1]
people[0].label_ == 'ORG'

False

In [73]:
people[0].text, len(people[0]

('Julian Assange', 2)

In [76]:
','.join([x.text for x in people if len(x) < 2])

'Models,Bestival,Photography,Cameron,Royal,Galloway,Men,Burberry,Cameron,Ant,Tights,Parks,Rosetta,Zara,Seinfeld,Older,Drama,Pornography,Rihanna,Royal,Biography,Burkini,Theresa,Depp,Slimane,Law'

In [77]:
others = 'Cameron,Royal,Galloway,Cameron,Seinfeld,Rihanna,Theresa,Depp,Slimane'

In [81]:
persons = [ x.text for x in people]
persons.extend(others.split(','))

In [82]:
bag = []

for i, kk in enumerate(kwords):
    for x in persons:
        if x in kk : 
            bag.append(i)
            break
            

['Seinfeld', 'Rihanna', 'Theresa', 'Depp', 'Slimane']

In [102]:
arts[0]

'I read somewhere that Blake Lively – who I guess is a person and not an exotic fruit – is to play a young Carrie Bradshaw. Didn\'t you say that the Sex and the City films had killed the franchise? How is this being allowed?  Mariella, I was as shocked as you. As you say, I made that ringing decree and read the last rites over the flailing corpse of Sex and the City and the series producer, Michael Patrick King, has wilfully – nay, foolishly – disregarded my command and is said to be planning a prequel to the series in which Ms Lively plays a young Carrie. Ah, prequels: sometimes good in theory, never good in practice. George Lucas, what are your thoughts on the matter?  While we await George\'s opinion, I need to stress two things: one, I personally think Blake Lively sounds more like the colour of a paint than an exotic fruit ("Yes, I like the idea of green for the kitchen but I don\'t want one that\'s too blake lively"). And two, this is a story that came from a terribly trashy maga

In [145]:
xx = textacy.preprocess_text(arts[0], lowercase=False, no_punct=True)
doc = textacy.doc.Doc(arts[0])

In [226]:
names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))
names = list(set(names))
names.sort(key = lambda x : -len(x))

nearly_unique = [ x.text for x in names if x.label_ == 'PERSON']
#monikers = [x for x in nearly_unique if ' ' not in x]
#set(nearly_unique)
nearly_unique = list(set(nearly_unique))
nearly_unique

['Sarah Jessica Parker',
 'Julian Assange',
 'King',
 'Gossip Girl',
 'George',
 'Kate Moss',
 'Mariella',
 'Carrie Bradshaw',
 "Osama bin Laden's",
 'Blake Lively – who',
 'Ding',
 'Philip',
 'Midas',
 'Michael Patrick King',
 'Andrea Dworkin-levels',
 'Jennifer Grey',
 'Carrie',
 'Blake Lively',
 'Tavi',
 'George Lucas']

# Replacement strategy

This is a greedy strategy.

1. try and replace the biggest string first
1. replace the substrings which should be forename, last name
1. finally look at what is left

In [201]:
replacers = []
for  x in nearly_unique:
    tt = x.split(' ')
    nn = len(tt) 
    if nn > 1:
        tt = x, x.split(' ')[0], x.split(' ')[-1]
    replacers.append(tt)
    
big_replacers = [x for x in replacers if len(x) > 1]
little_replacers = [x for x in replacers if len(x) == 1]

txt = arts[0]
for i, x in enumerate(big_replacers):
    for ss in x:
        txt = txt.replace(ss,'PERSON%d'%i)
        
total = i
for i, ss in enumerate(little_replacers):

    txt = txt.replace(ss[0],'PERSON%d'%(i+total) )


Now we'll deal with the other entities.

In [218]:
replacers2 = []
cats = ['WORK_OF_ART', 'ORG', 'GPE', 'PRODUCT']
tots = {x: 0 for x in cats}
for x in names:
    if x.label_ in cats:
        payload = x.text, '%s%d'%(x.label_, tots[x.label_])
        tots[x.label_] += 1
        replacers2.append(payload) 
        
for a,b in replacers2:
    txt = txt.replace(a,b )       



In [ ]:
trash = []

for base_txt in arts[:]:

    doc = textacy.doc.Doc(base_txt)
    txt = base_txt[:]

    names = list(textacy.extract.named_entities(doc, drop_determiners=True, exclude_types='numeric'))
    names = list(set(names))
    names.sort(key = lambda x : -len(x))

    nearly_unique = [ x.text for x in names if x.label_ == 'PERSON']


    replacers = []
    for  x in list(set(nearly_unique)):
        tt = x.split(' ')
        nn = len(tt) 
        if nn > 1:
            tt = x, x.split(' ')[0], x.split(' ')[-1]
        replacers.append(tt)

    big_replacers = [x for x in replacers if len(x) > 1]
    little_replacers = [x for x in replacers if len(x) == 1]

    # for some reason there was 'in' in the repacement queue
    for i, x in enumerate(big_replacers):
        for ss in x:
            if len(ss) < 3: continue
            txt = txt.replace(ss,'PERSON%d'%i)

    total = i
    for i, ss in enumerate(little_replacers):
        if len(ss) < 3: continue
        txt = txt.replace(ss[0],'PERSON%d'%(i+total) )


    replacers2 = []
    cats = ['WORK_OF_ART', 'ORG', 'GPE', 'PRODUCT']
    tots = {x: 0 for x in cats}
    for x in names:
        #need to fix this
        if len(x.text) < 3: continue
        if x.label_ in cats:
            payload = x.text, '%s%d'%(x.label_, tots[x.label_])
            tots[x.label_] += 1
            replacers2.append(payload) 

    for a,b in replacers2:
        txt = txt.replace(a,b ) 
    
    trash.append(txt)
        
    

In [ ]:
trash[-1]